Find unique location identifier in the data, to use that to find the record that corresponds to a particular school or university location. Then that walkability index can be used to draw up visualisations.


Just university and college tract and blocks only - no neighbourhood strategies...

Only educational institutions in Indiana... Got the table data from wikipedia, and then using the wikipedia page of that institution extract the coordinated and then use FCC Area API to get the FIPS. Using that FIPS and getting the data from the smartlocationdatabase csv file.

In [ ]:
!pip install geocoder

In [ ]:
from bs4 import BeautifulSoup
import csv
import geocoder
import html5lib
import json
import numpy as np
import openpyxl
import pandas as pd
import requests

In [ ]:
#Find the latest dataset at https://catalog.data.gov/dataset/walkability-index
df = pd.read_csv("/content/drive/MyDrive/Walkability Index/EPA_SmartLocationDatabase_V3_Jan_2021_Final.csv")
df.describe()

,OBJECTID,GEOID10,GEOID20,STATEFP,COUNTYFP,TRACTCE,BLKGRPCE,CSA,CBSA,CBSA_POP,...,D5DRI,D5DE,D5DEI,D2A_Ranked,D2B_Ranked,D3B_Ranked,D4A_Ranked,NatWalkInd,Shape_Length,Shape_Area
count,220740.000000,2.207390e+05,2.207400e+05,220740.000000,220740.000000,220740.000000,220740.000000,167709.000000,203645.000000,2.207400e+05,...,220740.000000,220740.000000,220740.000000,220740.000000,220740.000000,220740.000000,220740.000000,220740.000000,2.207400e+05,2.207400e+05
mean,110370.500000,2.870894e+11,2.870915e+11,28.623190,85.697449,262342.382110,2.221392,341.742661,30514.836210,3.607329e+06,...,-57326.486630,-57326.597590,-57326.483325,10.500000,10.500000,10.500000,7.624884,9.541628,1.655970e+04,4.466074e+07
std,63722.293548,1.640746e+11,1.640774e+11,16.386075,98.818946,351403.837442,1.195155,125.700068,10979.471375,5.219925e+06,...,49459.959391,49459.830782,49459.963223,5.766294,5.766294,5.766294,7.812853,4.373952,3.830373e+04,6.430513e+08
min,1.000000,1.001020e+10,1.001020e+10,1.000000,1.000000,100.000000,0.000000,104.000000,10100.000000,0.000000e+00,...,-99999.000000,-99999.000000,-99999.000000,1.000000,1.000000,1.000000,1.000000,1.000000,2.685713e+02,4.435890e+03
25%,55185.750000,1.312100e+11,1.312100e+11,13.000000,29.000000,10303.000000,1.000000,218.000000,19740.000000,2.053030e+05,...,-99999.000000,-99999.000000,-99999.000000,5.750000,5.750000,5.750000,1.000000,5.833333,3.271213e+03,4.985104e+05
50%,110370.500000,2.901900e+11,2.901900e+11,29.000000,61.000000,46298.000000,2.000000,361.000000,32820.000000,1.252890e+06,...,-99999.000000,-99999.000000,-99999.000000,10.500000,10.500000,10.500000,1.000000,9.166667,5.877323e+03,1.373918e+06
75%,165555.250000,4.200350e+11,4.200350e+11,42.000000,109.000000,482503.000000,3.000000,429.000000,39300.000000,4.673634e+06,...,0.142514,0.000354,0.161708,15.250000,15.250000,15.250000,16.000000,13.166667,1.596391e+04,9.413158e+06
max,220740.000000,7.803100e+11,7.803100e+11,78.000000,840.000000,993000.000000,9.000000,566.000000,49820.000000,1.931847e+07,...,1.000000,1.000000,1.000000,20.000000,20.000000,20.000000,20.000000,20.000000,3.925989e+06,1.641290e+11


In [ ]:
df.dtypes

OBJECTID          int64
GEOID10         float64
GEOID20         float64
STATEFP           int64
COUNTYFP          int64
                 ...   
D3B_Ranked        int64
D4A_Ranked        int64
NatWalkInd      float64
Shape_Length    float64
Shape_Area      float64
Length: 117, dtype: object


###Walkability Index

Label: 1 - 5.75 (Least Walkable)

Label: 5.76 - 10.50 (Below Average Walkable)

Label: 10.51 - 15.25 (Above Average Walkable)

Label: 15.26 - 20 (Most Walkable)

In [ ]:
#df[(df['STATEFP'] == 18) & (df['COUNTYFP'] == 105) & (df['TRACTCE'] == 202)]['NatWalkInd']

In [ ]:
'''
#https://stackoverflow.com/questions/25888396/how-to-get-latitude-longitude-with-python

def get_latlong(name):
  g = geocoder.bing(name, key='AhyTa3fHo0r5fI_9Ang8n0TXO9J77F8k0JmQWtg-VVUnsmXiPB-U4rwX5d_GXGtZ')
  results = g.json
  #print(results)
  return (results['lat'], results['lng'])
'''

In [ ]:
'''
#https://geo.fcc.gov/api/census/#!/block/get_block_find

latitude, longitude = get_latlong('Indiana University Bloomington')
print(latitude, longitude)

block_data = requests.get(f"https://geo.fcc.gov/api/census/block/find?latitude={str(latitude)}&longitude={str(longitude)}&censusYear=2020&showall=true&format=json").text
block_data = json.loads(block_data)
#print(block_data)
new_lat_long = block_data["Block"]['bbox']
print(new_lat_long)


block_data2 = get_api_data(f"https://geo.fcc.gov/api/census/block/find?latitude={str(new_lat_long[1])}&longitude={str(new_lat_long[0])}&censusYear=2020&showall=true&format=json").text
block_data2 = json.loads(block_data2)
print(block_data2)


state = int(block_data2["Block"]['FIPS'][0:2])
county = int(block_data2["Block"]['FIPS'][2:5])
tract = int(block_data2["Block"]['FIPS'][5:11])
print(state, county, tract)
'''

A 15-digit number that begins with the state identifier and ends with the block identifier. BLOCKID: 15-character code that is the concatenation of fields consisting of the 2-character state FIPS code, the 3-character county FIPS code, the 6-character census tract code, and the 4-character tabulation block code.

Cooridantes returned by BingAPI are not yeilding accurate values i.e. the data for corresponding block FIPS is not present in the smartlocationdatabase csv. Might have to depend on the boundary coordinates returned by the block api.

In [ ]:
#df[(df['STATEFP'] == state) & (df['COUNTYFP'] == county) & (df['TRACTCE'] == tract)]#['NatWalkInd']

In [ ]:
#List of all education institutions in Indiana... source: https://en.wikipedia.org/wiki/List_of_colleges_and_universities_in_Indiana

#df_edu_places = pd.read_csv("edu_list.csv")

Using beautiful soap to extract exact coordinates of the locations from their respective wikipedia page

In [ ]:
def scrape_lat_long(link):
  response = requests.get(link).text
  soup = BeautifulSoup(response, 'html.parser')
  coords=soup.find('span',"geo-dec").text
  lat_long = coords.split()
  lat = lat_long[0][:-2]
  long_ = '-' + lat_long[1][:-2]
  return(lat, long_)

In [ ]:
walkability_index_list = list()

df_edu_places_exl = pd.read_excel("Educational institutions in Indiana.xlsx")

wb = openpyxl.load_workbook("Educational institutions in Indiana.xlsx")
ws = wb['Sheet1']

for row in range(2, 64):
  cell = ws.cell(row=row, column=1)
  temp = [cell.value]
  try:
   latitude, longitude = scrape_lat_long(cell.hyperlink.target)
  except:
    #print(temp[0] + ' - No wikipedia link found.')
    pass

  block_data = requests.get(f"https://geo.fcc.gov/api/census/block/find?latitude={latitude}&longitude={longitude}&censusYear=2020&showall=true&format=json").text
  block_data = json.loads(block_data)

  state = int(block_data["Block"]['FIPS'][0:2])
  county = int(block_data["Block"]['FIPS'][2:5])
  tract = int(block_data["Block"]['FIPS'][5:11])

  temp.append(df[(df['STATEFP'] == state) & (df['COUNTYFP'] == county) & (df['TRACTCE'] == tract)]['NatWalkInd'].mean())
  if not np.isnan(temp[1]): walkability_index_list.append(temp) #Not considering nan value yielding lists



walkability_index_list.sort(key = lambda x: x[1])
print(walkability_index_list)

[['Saint Mary-of-the-Woods College', 4.083333333500001], ['Ancilla College', 4.2083333335], ['Rose-Hulman Institute of Technology', 4.2333333334], ['Indiana University East', 5.916666667], ['Oakland City University', 5.9666666666], ['Taylor University', 6.916666666749999], ['Hanover College', 7.111111111333333], ['Indiana University Northwest', 7.1111111115000005], ['Indiana University Kokomo', 7.333333333666666], ['Indiana University Southeast', 7.722222221], ['Purdue University Northwest', 7.722222222333333], ['Calumet College of St. Joseph', 7.75], ['Indiana University – Purdue University Columbus', 7.888888888666666], ['Ball State University', 8.333333333], ['Ottawa University–Jeffersonville', 8.3999999992], ['Wabash College', 8.4333333334], ['WGU Indiana', 8.4333333334], ['University of Evansville', 8.49999999925], ['Goshen College', 8.555555554333333], ['Trine University', 8.58333333425], ['Manchester University', 8.666666666000001], ['DePauw University', 9.4166666675], ['Bethel 

In [ ]:
#Nested lists to csv

with open('output.csv', 'w', newline='') as csvfile:
    writer = csv.writer(csvfile)
    writer.writerows(walkability_index_list)

For entire USA...

In order to obtain wikipedia_source.html, the following is done...

1. https://en.wikipedia.org/wiki/Lists_of_American_universities_and_colleges
2. In the above page, Tools -> What links here
3. Sorted list
4. Using inspect, copy the outer html of the html element.
5. Save the code to a file i.e., wikipedia_source.html


In [ ]:
#Extract all links from the wikipedia page to get the links for the list of colleges in each state

#Lists of American universities and colleges - wikipedia
with open('/content/drive/MyDrive/Walkability Index/wikipedia_source.html', 'r') as f:
  contents = f.read()
  soup = BeautifulSoup(contents, 'lxml')
  #print(soup)
  state_unis = {}
  for link in soup.find_all("a"):
      url = link.get("href", "")
      if "/wiki/List_of_colleges_and_universities_in_" in url:
          state_unis[link.text.strip()] = url

'''
#dictionary to csv
with open('state_unis_urls.csv', 'w') as csvfile:
    writer = csv.writer(csvfile)
    writer.writerow(['Link text', 'Link'])
    count = 0
    for key, value in state_unis.items():
      count += 1
      writer.writerow([key, value])
      if count == 54: break
'''

In [ ]:
def strip_string(stri):
  if stri[-3:] == '[c]': return stri[:-3]
  else: return stri

In [ ]:
#list of colleges and universities in a state

walkability_index_list = list()
wiki_begin = 'https://en.wikipedia.org'
count = 0

for value in state_unis.values():

  count += 1
  #get the table containing the details of all the colleges and universities in a state
  df_state_unis = pd.read_html(value, extract_links="all")[0]
  print(f"Progress: {count}/{len(state_unis)}", value)

  for uni in df_state_unis.iloc[:, 0]: #looping over institution names and their wikis
    temp = [strip_string(uni[0])]
    try:
      url = wiki_begin + uni[1]
      latitude, longitude = scrape_lat_long(url)
    except:
      #print(temp[0] + ' - No wikipedia link found.')
      continue

    block_data = requests.get(f"https://geo.fcc.gov/api/census/block/find?latitude={latitude}&longitude={longitude}&censusYear=2020&showall=true&format=json").text
    block_data = json.loads(block_data)

    state = int(block_data["Block"]['FIPS'][0:2])
    county = int(block_data["Block"]['FIPS'][2:5])
    tract = int(block_data["Block"]['FIPS'][5:11])

    try:
      temp.append(df[(df['STATEFP'] == state) & (df['COUNTYFP'] == county) & (df['TRACTCE'] == tract)]['NatWalkInd'].mean())
      #print(temp)
    except:
      continue

    if not np.isnan(temp[1]): walkability_index_list.append(temp) #Not considering nan value yielding lists

  if count == 54: break

walkability_index_list.sort(key = lambda x: x[1])
print(walkability_index_list)

Progress: 1/60 https://en.wikipedia.org/wiki/List_of_colleges_and_universities_in_Alabama
Progress: 2/60 https://en.wikipedia.org/wiki/List_of_colleges_and_universities_in_Alaska
Progress: 3/60 https://en.wikipedia.org/wiki/List_of_colleges_and_universities_in_Arizona
Progress: 4/60 https://en.wikipedia.org/wiki/List_of_colleges_and_universities_in_Arkansas
Progress: 5/60 https://en.wikipedia.org/wiki/List_of_colleges_and_universities_in_Baltimore
Progress: 6/60 https://en.wikipedia.org/wiki/List_of_colleges_and_universities_in_California
Progress: 7/60 https://en.wikipedia.org/wiki/List_of_colleges_and_universities_in_Colorado
Progress: 8/60 https://en.wikipedia.org/wiki/List_of_colleges_and_universities_in_Connecticut
Progress: 9/60 https://en.wikipedia.org/wiki/List_of_colleges_and_universities_in_Delaware
Progress: 10/60 https://en.wikipedia.org/wiki/List_of_colleges_and_universities_in_Florida
Progress: 11/60 https://en.wikipedia.org/wiki/List_of_colleges_and_universities_in_Georg

In [ ]:
#Nested lists to csv
def list_to_csv(filename, list_data):
  with open(filename, 'w', newline='') as csvfile:
      writer = csv.writer(csvfile)
      writer.writerow(['column' + str(i) for i in range(len(list_data[0]))]) #default column names
      writer.writerows(list_data)

list_to_csv('output.csv', walkability_index_list)

In [ ]:
#group the institutions based on the walkability index for visualisations

#Label: 1 - 5.75 (Least Walkable)
#Label: 5.76 - 10.50 (Below Average Walkable)
#Label: 10.51 - 15.25 (Above Average Walkable)
#Label: 15.26 - 20 (Most Walkable)

least_walkable, bavg_walkable, aavg_walkable, most_walkable = [], [], [], []

for list_ in walkability_index_list: #list_ has institution name and its respective walkability index
  if list_[1] <= 5.75:
    least_walkable.append(list_)
  elif list_[1] <= 10.5:
    bavg_walkable.append(list_)
  elif list_[1] <= 15.25:
    aavg_walkable.append(list_)
  else:
    most_walkable.append(list_)

#creating csv files for each of the group
list_to_csv('least_walkable.csv', least_walkable)
list_to_csv('bavg_walkable.csv', bavg_walkable)
list_to_csv('aavg_walkable.csv', aavg_walkable)
list_to_csv('most_walkable.csv', most_walkable)

!! How to tackle the surrounding blocks problem, should the neighbouring blocks be considered or not...

if considered, certain blocks are very large in area compared to others and including their neighbours would bring in walkability index that might not be relevant.

some blocks are small and not considering neighbours could mean a very perspective from walkability index and only a part of the university might be in that location while other parts may be in other blocks.